# Imports

In [10]:
# from kfp import dsl
# from kfp.v2 import compiler
from google.cloud import aiplatform
from typing import List, Dict
import boto3
from botocore.exceptions import ClientError
import json, os, ast, re
import logging
from datetime import datetime, timedelta
import pandas as pd, numpy as np
from scipy.special import softmax
from pydantic import BaseModel, Field, ValidationError

import scrubadub, scrubadub_spacy

import snowflake.connector as sc
from snowflake.connector.pandas_tools import write_pandas

import vertexai
import vertexai.preview.generative_models as generative_models
from vertexai.generative_models import GenerativeModel, GenerationConfig, Part

# Sentiments
from transformers import pipeline
from transformers import AutoTokenizer, AutoConfig
from transformers import AutoModelForSequenceClassification

# Variables

In [11]:
# Temporary secrets manager
with open("configs.json", 'r') as secrets_file:
    configs = json.load(secrets_file)

loc_logs = configs.get("loc_logs")
excel_path = configs.get("excel_path")

aws_access_key = configs.get("aws_access_key")
aws_secret_key = configs.get("aws_secret_key")

# AWS
s3_source_bucket = configs.get('s3_source_bucket')
s3_transcripts_location = configs.get('s3_transcripts_location')

# GCP
gcp_project_id=configs.get('gcp_project_id')
gcp_prjct_location=configs.get('gcp_prjct_location')

# Snowflake
private_key_file = configs.get('snowflakegcp_rsa_key')
private_key_file_pwd = configs.get('snf_ssh_key_pass')

conn_params = {
    'account': configs.get('snf_account'),
    'user': configs.get('snf_user'),
    'private_key_file': configs.get('snf_private_key_file'),
    'private_key_file_pwd':configs.get('snf_private_key_pwd'),
    'warehouse': configs.get('snf_warehouse'),
    'database': configs.get('snf_database'),
    'schema': configs.get('snf_schema')
}

# # Sentiment Scores
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model_sentiment = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Util Functions

## Misc Utils

### Initiate Master Inter and Intra Dataframes

In [12]:
def initiate_master_dataframes():
    if os.path.isfile("df_intra_calls_data.csv"):
        logger.info("df_intra_calls_data.csv exists.") 
        df_intra_calls_data = pd.read_csv("df_intra_calls_data.csv")
        df_intra_calls_data.CONTACT_ID = df_intra_calls_data.CONTACT_ID.astype('string')
    else:
        logger.info("df_intra_calls_data.csv does not exists.")
        df_intra_calls_data = pd.DataFrame()

    if os.path.isfile("df_inter_calls_data.csv"):
        logger.info("df_inter_calls_data.csv exists.")
        df_inter_calls_data = pd.read_csv("df_inter_calls_data.csv")
        df_inter_calls_data.CONTACT_ID = df_inter_calls_data.CONTACT_ID.astype('string')
    else:
        logger.info("df_inter_calls_data.csv does not exists.")
        df_inter_calls_data = pd.DataFrame()

    return df_intra_calls_data, df_inter_calls_data

## Function: Listing Transcripts

In [13]:
def list_new_transcripts(aws_access_key: str, aws_secret_key: str, source_bucket: str, custom_location: str, max_objects: int):
    """
    Fetch audio file from S3 and return it as a BytesIO object
    """
    try:
        s3_client = boto3.client(
            's3',
            aws_access_key_id=aws_access_key,
            aws_secret_access_key=aws_secret_key
        )

        logger.info("")
        logger.info("Fetching New Transcripts to process")
        # List files in the folder
        response = s3_client.list_objects_v2(Bucket=s3_source_bucket, Prefix=s3_transcripts_location)
        
        list_transcripts = []
        for obj in response.get('Contents', []):
            if obj['Key'].endswith('.json'):
                list_transcripts.append([obj['Key'], obj['LastModified']])
                logger.info(f"{str(obj['LastModified']) +": "+ obj['Key']}")
                if len(list_transcripts) >= max_objects:
                    break  # Exit the loop after printing max objects
    
        return list_transcripts

    except ClientError as e:
        logger.error(f"Error accessing S3: {e}")
        pass

## Function: Read Transcripts

In [14]:
def fetch_transcript_from_s3(aws_access_key: str, aws_secret_key: str, s3_source_bucket: str, file_key):
    """
    Read Transcript JSON content from a specific file in S3.
    
    :param bucket_name: Name of the S3 bucket
    :param file_key: Full path/key of the JSON file
    :return: Parsed JSON content
    """
    s3_client = boto3.client(
            's3',
            aws_access_key_id=aws_access_key,
            aws_secret_access_key=aws_secret_key
        )
    
    try:
        # Download the file
        response = s3_client.get_object(Bucket=s3_source_bucket, Key=file_key)
        
        # Read the content
        json_content = response['Body'].read().decode('utf-8')
        
        # Parse JSON
        return json.loads(json_content)
    
    except Exception as e:
        logger.info
        logger.error(f"Error reading Transcript JSON file {file_key}: {e}")
        logger.info("")
        pass

### Handel Duplicates

In [52]:
def handle_duplicates(data_frame, columns_to_check):
    """
    Dedupes the final Dataframes to be written to the Snowflake Tables
    """
    # Remove duplicate records based on the specified columns, keeping the first occurrence
    df_cleaned = data_frame.drop_duplicates(subset=columns_to_check, keep="first")

    # (Optional) Reset index after removing duplicates
    df_cleaned = df_cleaned.reset_index(drop=True)

    return df_cleaned


# Create Intra-call Dataframe

In [15]:
def millis_to_hhmmss(millis):
    """Convert milliseconds to mm:ss format"""
    total_seconds = int(millis / 1000)
    hours = total_seconds // 3600
    minutes = total_seconds // 60
    seconds = total_seconds % 60
    return f"{minutes:02d}:{seconds:02d}"

def convert_to_seconds(time_str):
    try:
        # Parse time string using datetime
        time_obj = datetime.strptime(time_str, '%H:%M:%S')
        # Convert to timedelta and extract total seconds
        total_seconds = time_obj.minute * 60 + time_obj.second
        return total_seconds
    except ValueError:
        pass

def process_transcript(
    transcript_data: dict,
    contact_id: str
):
    """
    Pre-process the transcript loaded from S3 Buckets:
    1. Load the transcript as Pandas Dataframe.
    2. Select only the necessary columns ['BeginOffsetMillis', 'EndOffsetMillis', 'ParticipantId', 'Content', 'Sentiment', 'LoudnessScore'].
    3. Format the time in minutes and seconds.
    4. Rename the columns for better understanding.
    """
    # Load the Transcript as Pandas Dataframe
    transcript_df = pd.json_normalize(transcript_data['Transcript'])

    # Select the relevant Columns
    columns_to_select = [
        'BeginOffsetMillis',
        'EndOffsetMillis',
        'ParticipantId',
        'Content'
    ]
    formatted_df = transcript_df[columns_to_select].copy()
    
    # Optionally rename columns to reflect their new format
    formatted_df = formatted_df.rename(columns={
        'BeginOffsetMillis': 'Begin_Offset',
        'EndOffsetMillis': 'End_Offset',
        'Content': 'caption',
        'Sentiment': 'sentiment_label',
        'ParticipantId': 'speaker_tag'
    })

    # Inserting the Call ID:
    formatted_df.insert(loc=0, column='contact_id', value=contact_id)
    formatted_df['call_language'] = transcript_data['LanguageCode']

    return formatted_df

def get_sentiment_label(row):
    # Check conditions in order of priority (Positive > Negative > Neutral)
    if row['positive'] > row['negative'] and row['positive'] > row['neutral']:
        return 'Positive'
    elif row['negative'] > row['positive'] and row['negative'] > row['neutral']:
        return 'Negative'
    else:
        return 'Neutral'

def get_sentiment_scores(text_list):
    dict_sentiments = []
    for text in text_list:
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model_sentiment(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = np.round(np.multiply(softmax(scores), 100), 2)
        merged_dict = dict(zip(list(config.id2label.values()), list(scores)))
        dict_sentiments.append(merged_dict)

    df_dict_sentiments = pd.DataFrame(dict_sentiments)
    df_dict_sentiments['sentiment_lable'] = df_dict_sentiments[['positive','negative','neutral']].apply(get_sentiment_label, axis=1)
    
    return df_dict_sentiments

def get_different_times(intra_call):
    # Apply formatting to both time columns
    intra_call['start_time_second'] = (intra_call['Begin_Offset'] / 1000).astype(int)
    # intra_call['Begin_Offset'] = intra_call['Begin_Offset'].apply(millis_to_hhmmss)
    intra_call['end_time_second'] = (intra_call['End_Offset'] / 1000).astype(int)
    # intra_call['End_Offset'] = intra_call['End_Offset'].apply(millis_to_hhmmss)
    intra_call['time_spoken_second'] = intra_call['end_time_second'] - intra_call['start_time_second']
    intra_call['time_spoken_second'] = intra_call['time_spoken_second'].where(intra_call['time_spoken_second'] >= 0, 0)
    intra_call['time_spoken_second'] = intra_call['time_spoken_second'].fillna(0).astype(int)
    intra_call['time_silence_second'] = intra_call['start_time_second'].shift(-1) - intra_call['end_time_second']
    intra_call['time_silence_second'] = intra_call['time_silence_second'].where(intra_call['time_silence_second'] >= 0, 0)
    intra_call['time_silence_second'] = intra_call['time_silence_second'].fillna(0).astype(int)
    intra_call['load_date'] = datetime.now()

    # Dropping time formatted columns
    intra_call = intra_call.drop(['Begin_Offset', 'End_Offset'], axis=1)

    return intra_call
    
def create_intra_call_df(aws_access_key: str, aws_secret_key: str, transcript_data: dict, contact_id: str):
    try:  
        # Get the relevant columns from the loaded transcript file
        intra_call = process_transcript(transcript_data, contact_id)
        
        df_sentiment_scores = get_sentiment_scores(intra_call.caption.to_list())
        intra_call = pd.concat([intra_call, df_sentiment_scores], axis=1)    
        intra_call = get_different_times(intra_call)    
        
        return intra_call
        
    except Exception as e:
        logger.info
        logger.error(f"{contact_id}: Error Creating Intra Call df: {e}")
        logger.info("")
        pass

# Create Inter-call Dataframe

In [16]:
class CategoryValidator:
    def __init__(self, excel_path: str):
        """Initialize with category mapping from Excel"""
        self.category_mapping = pd.read_excel(excel_path, sheet_name='Vlook')
        self.valid_categories = set(self.category_mapping['Category'].unique())
        self.category_subcategory_map = self._create_category_mapping()

    def _create_category_mapping(self):
        """Create category to subcategory mapping"""
        mapping = {}
        for _, row in self.category_mapping.iterrows():
            if row['Category'] not in mapping:
                mapping[row['Category']] = set()
            mapping[row['Category']].add(row['SubCategory'])
        return mapping

    def validate_category(self, category: str) -> bool:
        """Check if category is valid"""
        return category in self.valid_categories

    def validate_subcategory(self, category: str, subcategory: str) -> bool:
        """Check if subcategory is valid for given category"""
        return category in self.category_subcategory_map and subcategory in self.category_subcategory_map[category]

    def get_valid_subcategories(self, category: str):
        """Get valid subcategories for a category"""
        return self.category_subcategory_map.get(category, set())

class CallSummary(BaseModel):
    summary: str = Field(..., max_length=500)
    key_points: List[str] = Field(..., max_items=5)
    outcome: str = Field(..., max_length=200)
    follow_up_recommendations: List[str] = Field(..., max_items=3)

class CallTopic(BaseModel):
    primary_topic: str = Field(..., max_length=100)
    category: str = Field(..., max_length=100)
    sub_category: str = Field(..., max_length=100)

    def validate_category_mapping(self, category_validator: CategoryValidator) -> bool:
        """Validate category and subcategory against mapping"""
        if not category_validator.validate_category(self.category):
            logger.error(f"Invalid category: {self.category}")
        if not category_validator.validate_subcategory(self.category, self.sub_category):
            logger.error(f"Invalid subcategory '{self.sub_category}' for category '{self.category}'")

class AgentCoaching(BaseModel):
    strengths: List[str] = Field(..., max_items=3)
    improvement_areas: List[str] = Field(..., max_items=3)
    specific_recommendations: List[str] = Field(..., max_items=4)
    skill_development_focus: List[str] = Field(..., max_items=3)

class TranscriptAnalysis(BaseModel):
    call_summary: CallSummary
    call_topic: CallTopic
    agent_coaching: AgentCoaching

class KPIExtractor:
    def __init__(self, project_id: str, location: str, excel_path: str):
        vertexai.init(project=project_id, location=location)
        self.model = GenerativeModel("gemini-1.5-flash-002")
        self.category_validator = CategoryValidator(excel_path)
        
        self.generation_config = {
            "temperature": 0.3,
            "max_output_tokens": 1024,
            "top_p": 0.8,
            "top_k": 40,
            "response_format": "json"
        }
        
        self.safety_settings = {
            generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        }

    def get_categories_prompt(self) -> str:
        """Create prompt section for valid categories and subcategories"""
        categories_prompt = []
        for category, subcategories in self.category_validator.category_subcategory_map.items():
            subcats = ', '.join(sorted(subcategories))
            categories_prompt.append(f"Category '{category}' can have subcategories: {subcats}")
        return '\n'.join(categories_prompt)

    def create_prompt(self, transcript: str) -> str:
        """Create structured prompt with category guidance"""
        categories_guidance = self.get_categories_prompt()
        
        return f"""
        Analyze this call transcript and provide a structured analysis in the exact JSON format specified below.
        Keep responses concise, specific, and actionable.

        Guidelines:
        - Call summary should be factual and highlight key interactions
        - Topics and categories MUST match the following valid mappings:
        {categories_guidance}
        - Coaching points should be specific and actionable
        - All responses must follow the exact structure specified
        - Ensure all lists have the specified maximum number of items
        - All text fields must be clear, professional, and free of fluff

        Transcript:
        {transcript}

        Required Output Structure:
        {{
            "call_summary": {{
                "summary": "3-4 line overview of the call",
                "key_points": ["Point 1", "Point 2", "Point 3", "Point 4", "Point 5"],
                "outcome": "Brief description of call outcome",
                "follow_up_recommendations": ["Rec 1", "Rec 2", "Rec 3"]
            }},
            "call_topic": {{
                "primary_topic": "Main topic of discussion",
                "category": "MUST BE ONE OF THE VALID CATEGORIES LISTED ABOVE",
                "sub_category": "MUST BE A VALID SUB-CATEGORY FOR THE CHOSEN CATEGORY"
            }},
            "agent_coaching": {{
                "strengths": ["Strength 1", "Strength 2", "Strength 3"],
                "improvement_areas": ["Area 1", "Area 2", "Area 3"],
                "specific_recommendations": ["Rec 1", "Rec 2", "Rec 3", "Rec 4"],
                "skill_development_focus": ["Skill 1", "Skill 2", "Skill 3"]
            }}
        }}

        Rules:
        1. Maintain exact JSON structure
        2. No additional fields or comments
        3. No markdown formatting
        4. Ensure all arrays have the exact number of items specified
        5. Keep all text concise and professional
        6. Do not mention any PII information such as Customer Name etc.
        7. STRICTLY use only the categories and subcategories from the provided mapping
        """

    def extract_json(self, response: str) -> Dict:
        """Extract valid JSON from response"""
        match = re.search(r'```json\s*([\s\S]*?)\s*```', response)
        if match:
            json_str = match.group(1)
        else:
            json_str = response.strip()
        
        try:
           return json.loads(json_str)
        except json.JSONDecodeError:
           logger.error("Invalid JSON response")
           pass

    def validate_response(self, response_json: Dict, contact_id: str = None) -> TranscriptAnalysis:
        """Validate response using Pydantic models and category mapping"""
        try:
            # First validate basic structure with Pydantic
            analysis = TranscriptAnalysis(**response_json)
            
            # Then validate category mapping
            analysis.call_topic.validate_category_mapping(self.category_validator)
            
            return analysis
        except ValidationError as e:
            logger.error(f"{contact_id if contact_id else ''}: Pydantic validation error - {e}")
            pass
        except ValueError as e:
            logger.error(f"{contact_id if contact_id else ''}: Category validation error - {e}")
            pass

    def extract_genai_kpis(self, transcript: str, contact_id: str = None) -> Dict:
        """Extract KPIs from transcript with validation"""
        try:
            # Generate prompt
            prompt = self.create_prompt(transcript)
            
            # Get response from Gemini
            response = self.model.generate_content(
                prompt
                # generation_config=self.generation_config,
                # safety_settings=self.safety_settings
            )
            
            # Parse JSON response
            response_json = self.extract_json(response.text)
            
            # Validate response structure and categories
            validated_response = self.validate_response(response_json, contact_id)
            
            return validated_response.model_dump()
            
        except Exception as e:
            logger.error(f"{contact_id if contact_id else ''}: Error extracting KPIs: {str(e)}")
            pass

def dict_to_newline_string(data: dict) -> str:
    """Converts a dictionary into a new-line formatted string."""
    formatted_str = ""
    for key, value in data.items():
        formatted_str += f"{key}:\n"
        for item in value:
            formatted_str += f"  - {item}\n"
    return formatted_str.strip()
    
def create_inter_call_df(
    gcp_project_id: str,
    gcp_prjct_location: str,
    df_intra_call: pd.DataFrame,
    transcript_data: dict,
    ac_last_modified_date: datetime,
    excel_path: str
):
    try:
        contact_id = df_intra_call.contact_id.unique
        # Redact PII Data
        # logger.info(f"{contact_id}: Redacting PII data.")
        scrubber = scrubadub.Scrubber()
        scrubber.add_detector(scrubadub_spacy.detectors.SpacyEntityDetector)
        df_intra_call.caption = df_intra_call.caption.apply(scrubber.clean)
    
        # logger.info(f"{contact_id}: Extract KPIs from Gemini")      
        extractor = KPIExtractor(gcp_project_id, gcp_prjct_location, excel_path)
        transcript = " ".join(df_intra_call.caption)
        call_gen_kpis = extractor.extract_genai_kpis(transcript)
        # logger.info(f"{contact_id}: Completed Extracting KPIs from Gemini") 
    
        # logger.info(f"{contact_id}: Creating Inter Call df")
        inter_call_dict = {}
        inter_call_dict['contact_id'] = str(df_intra_call['contact_id'][0])
        inter_call_dict['call_text'] = " ".join(df_intra_call.caption)
        inter_call_dict['call_summary'] = call_gen_kpis['call_summary']['summary']
        inter_call_dict['topic'] = call_gen_kpis['call_topic']['primary_topic']
        inter_call_dict['category'] = call_gen_kpis['call_topic']['category']
        inter_call_dict['category_generated'] = call_gen_kpis['call_topic']['category']
        inter_call_dict['sub_category'] = call_gen_kpis['call_topic']['sub_category']
        inter_call_dict['sub_category_generated'] = call_gen_kpis['call_topic']['sub_category']
        inter_call_dict['agent_coaching'] = dict_to_newline_string(call_gen_kpis['agent_coaching'])
    
        df_inter_call = pd.DataFrame(pd.Series(inter_call_dict)).T
    
        # Add metadata from AWS
        # df_inter_call['account_id'] = transcript_data['AccountId']
        df_inter_call['agent_speech_speed'] = transcript_data['ConversationCharacteristics']['TalkSpeed']['DetailsByParticipant']['AGENT']['AverageWordsPerMinute']
        df_inter_call['customer_speech_speed'] = transcript_data['ConversationCharacteristics']['TalkSpeed']['DetailsByParticipant']['CUSTOMER']['AverageWordsPerMinute']
        df_inter_call['total_talktime_agent_second'] = int(transcript_data['ConversationCharacteristics']['TalkTime']['DetailsByParticipant']['AGENT']['TotalTimeMillis']/1000)
        df_inter_call['total_talktime_customer_second'] = int(transcript_data['ConversationCharacteristics']['TalkTime']['DetailsByParticipant']['CUSTOMER']['TotalTimeMillis']/1000)
        df_inter_call['total_talktime_call_second'] = int(transcript_data['ConversationCharacteristics']['TalkTime']['TotalTimeMillis']/1000)
        df_inter_call['total_duration_call_second'] = int(transcript_data['ConversationCharacteristics']['TotalConversationDurationMillis']/1000)
        df_inter_call['total_dead_air_call_second'] = df_inter_call['total_duration_call_second'] - df_inter_call['total_talktime_call_second']
        # df_inter_call['customer_instance_id'] = transcript_data['CustomerMetadata']['InstanceId']
        # df_inter_call['call_job_status'] = transcript_data['JobStatus']
        df_inter_call['call_language'] = transcript_data['LanguageCode']
        df_inter_call['call_s3_uri'] = transcript_data['CustomerMetadata']['InputS3Uri']
        df_inter_call['ac_last_modified_date'] = ac_last_modified_date
        df_inter_call['load_date'] = datetime.now()
        
        return df_inter_call

    except Exception as e:
        logger.info
        logger.error(f"{contact_id}: Error Creating Intra Call df: {e}")
        logger.info("")
        pass

# Writing Dataframe to Snowflake

In [27]:
def insert_new_records(conn, table_name, df):
    """
    Inserts only new records (based on ID) into Snowflake table.
    1. Fetches existing IDs from table
    2. Filters out rows with existing IDs from DataFrame
    3. Inserts only new records
    
    Args:
        conn: Snowflake connection object
        table_name: Name of the target table
        df: Pandas DataFrame containing the data (must have 'id' column)
    """
    logger.info(f"Writing data to table: {table_name}")
    
    cursor = conn.cursor()
    
    # Get existing IDs from Snowflake table
    cursor.execute(f"SELECT DISTINCT(CONTACT_ID) FROM {table_name}")
    existing_ids = {row[0] for row in cursor.fetchall()}
    
    # Filter DataFrame to keep only new records
    new_records_df = df[~df['CONTACT_ID'].isin(existing_ids)]
    
    if len(new_records_df) == 0:
        logger.info("No new records to insert")
        return 0
    
    # Insert new records
    success, nchunks, nrows, _ = write_pandas(conn, new_records_df, table_name)
    
    logger.info(f"Inserted {nrows} new records")
    logger.info(f"Skipped {len(df) - len(new_records_df)} existing records")
    
    cursor.close()
    return nrows

# Logging Handling

In [21]:
def setup_logger(log_file):
    """
    Sets up a logger that writes to both file and console with timestamp.

    Args:
        log_file (str): Name of the log file to write to.

    Returns:
        logger: Configured logger instance.
    """
    logger = logging.getLogger('voice_ai_logger')

    # Reset handlers if already exist
    if logger.hasHandlers():
        logger.handlers.clear()

    # Set log level
    logger.setLevel(logging.DEBUG)

    # Create formatter
    formatter = logging.Formatter(
        '%(asctime)s [%(levelname)s]: %(message)s', datefmt='%Y-%m-%d %H:%M:%S'
    )

    # Create file handler
    file_handler = logging.FileHandler(log_file)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    # Create a stream handler (console)
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

    return logger


def reset_logging():
    """Removes all logging handlers and resets the logger."""
    # Get all loggers
    loggers = list(logging.root.manager.loggerDict.values())
    
    # Include root logger explicitly
    loggers.append(logging.getLogger())

    for logger in loggers:
        if isinstance(logger, logging.Logger):  # Ensure it's a valid logger instance
            for handler in logger.handlers[:]:  # Copy list to avoid modification issues
                logger.removeHandler(handler)
                handler.close()

            logger.setLevel(logging.NOTSET)
            logger.propagate = False  # Avoid duplicate logs from propagation

# Main Function

In [54]:
reset_logging()

# Setup logger
log_file='voice_ai_runtime.logs'
logger = setup_logger(log_file)

# Initiating Master DataFrames
logger.info("Called: Initiate Master Dataframes")
df_intra_calls_data, df_inter_calls_data = initiate_master_dataframes()

# Get the transcripts in to_process_folder
max_objects = 50
list_transcripts = list_new_transcripts(aws_access_key, aws_secret_key, s3_source_bucket, s3_transcripts_location, max_objects)

logger.info("")
logger.info(f"Transcripts to process: {len(list_transcripts)}")
logger.info("")

# If there are transcripts to be processed
if len(list_transcripts) == 0:
    logger.info("No Transcripts to Process")
    logger.info("")

else:
    # Process the call
    for transcript in list_transcripts:
        # get the call ID
        contact_id = transcript[0].split('/')[-1].split('.')[0].split('analysis')[0].strip('_')
        ac_last_modified_date = datetime.strptime(transcript[0].split('analysis_')[-1].split('.')[0].replace('_', ':'), '%Y-%m-%dT%H:%M:%SZ')

        # Check if Call Already Processed
        if (len(df_intra_calls_data) > 0 and contact_id in df_intra_calls_data.CONTACT_ID.unique()) and (len(df_inter_calls_data) > 0 and contact_id in df_inter_calls_data.CONTACT_ID.unique()):
            logger.info(f"{contact_id}: Call already Processed.")
            logger.info("")
            logger.info("")
            # break

        else:
            # get the audio transcript file name
            logger.info(f"{contact_id}: Processing") 
            transcript_file = transcript[0]
            
            # Get the Transcript file from S3 Bucket
            logger.info(f"{contact_id}: Fetching Transcript from S3")
            transcript_data = fetch_transcript_from_s3(aws_access_key, aws_secret_key, s3_source_bucket, transcript_file)
            logger.info(f"{contact_id}: Successfully fetched the Transcript from S3")
        
            # Create the Inter Call KPIs
            logger.info(f"{contact_id}: Creating df_intra_call ")
            df_intra_call = create_intra_call_df(aws_access_key, aws_secret_key, transcript_data, contact_id)
            logger.info(f"{contact_id}: Successfully created df_intra_call ")
        
            # Create the Intra Call KPIs
            logger.info(f"{contact_id}: Creating df_inter_call ")
            df_inter_call = create_inter_call_df(gcp_project_id, gcp_prjct_location, df_intra_call, transcript_data, ac_last_modified_date, excel_path)
            logger.info(f"{contact_id}: Successfully created df_inter_call ")

            ###============================================================###
            # Save DataFrames only when both are having data
            if not df_intra_call.empty and not df_inter_call.empty:
                # Appending to Intra-calls Master DataFrame
                df_intra_call.columns = df_intra_call.columns.str.upper()  # Capitalising Column names for Snowflake
                df_intra_calls_data = pd.concat([df_intra_calls_data, df_intra_call], ignore_index=True)
                df_intra_calls_data.to_csv("df_intra_calls_data.csv", index=False)
                logger.info(f"{contact_id}: Persisted df_intra_calls_data to CSV.")
                
                # Appending to Inter-calls Master DataFrame
                df_inter_call.columns = df_inter_call.columns.str.upper()  # Capitalising Column names for Snowflake
                df_inter_calls_data = pd.concat([df_inter_calls_data, df_inter_call], ignore_index=True)
                df_inter_calls_data.to_csv("df_inter_calls_data.csv", index=False)
                logger.info(f"{contact_id}: Persisted df_intra_calls_data to CSV.")
                logger.info(f"{contact_id}: Processing Complete")
                logger.info("")
                logger.info("")
                
            else:
                if df_intra_call.empty:
                    logger.error("Intra Call DataFrame was not created successfully.")
                if df_inter_call.empty:
                    logger.error("Inter Call DataFrame was not created successfully.")
                    

    logger.info(f"Removing duplicates in df_inter_calls_data.")
    columns_to_check = ["CONTACT_ID"]
    df_inter_calls_data = handle_duplicates(df_inter_calls_data, columns_to_check)
    logger.info(f"Removing duplicates in df_intra_calls_data.")
    columns_to_check = ["CONTACT_ID", "SPEAKER_TAG", "CAPTION", "START_TIME_SECOND", "END_TIME_SECOND"]
    df_intra_calls_data = handle_duplicates(df_intra_calls_data, columns_to_check)
    

    logger.info(f"Writing Dataframe to Snowflake.")
    conn = sc.connect(**conn_params)
    table_name ='SRC_GCP_INTER_CALLS'
    insert_new_records(conn, table_name, df_inter_calls_data)
    
    table_name ='SRC_GCP_INTRA_CALLS'
    insert_new_records(conn, table_name, df_intra_calls_data)
    conn.close()

    logger.info(f"Completed processing {len(list_transcripts)} Calls")

2025-02-24 18:22:59 [INFO]: Called: Initiate Master Dataframes
2025-02-24 18:22:59 [INFO]: df_intra_calls_data.csv exists.
2025-02-24 18:22:59 [INFO]: df_inter_calls_data.csv exists.
2025-02-24 18:22:59 [INFO]: 
2025-02-24 18:22:59 [INFO]: Fetching New Transcripts to process
2025-02-24 18:23:00 [INFO]: 2025-02-20 12:21:53+00:00: connect-audio-files/to_process_transcripts/0598ed6e-e088-4740-b8ca-1a56b987ae8b_analysis_2025-01-29T18_28_42Z.json
2025-02-24 18:23:00 [INFO]: 2025-02-20 12:21:59+00:00: connect-audio-files/to_process_transcripts/0a21616f-c523-4327-bffd-0b6c24335f7d_analysis_2025-01-29T14_34_27Z.json
2025-02-24 18:23:00 [INFO]: 2025-02-20 12:22:57+00:00: connect-audio-files/to_process_transcripts/309a0db8-2735-4537-988a-a66bff37c159_analysis_2025-01-17T21_06_34Z.json
2025-02-24 18:23:00 [INFO]: 2025-02-20 12:22:58+00:00: connect-audio-files/to_process_transcripts/3c2ddea2-d6e3-4acd-8abd-6eb98c192bd8_analysis_2025-01-06T18_29_29Z.json
2025-02-24 18:23:00 [INFO]: 2025-01-30 09:05

No new records to insert


2025-02-24 18:23:09 [INFO]: Inserted 2777 new records
2025-02-24 18:23:09 [INFO]: Skipped 0 existing records
2025-02-24 18:23:09 [INFO]: Completed processing 31 Calls
